## **Solar Photovoltaic Generation in Spain:** Time Series Modeling and Forecast (2015-2027)
### **1. Introduction**
#### **Objective**
This project provides a detailed analysis of the evolution of renewable energy in Spain over the past decade, with a specific focus on the growth of electricity generated from solar photovoltaic (PV) installations. The study evaluates time series modeling techniques to project future solar PV output and assess how this energy source may evolve in the coming years. These forecasts offer valuable insight into the potential trajectory of solar generation in Spain and support long-term planning and policy decisions in the renewable energy sector.
#### **Dataset description**
This dataset records the monthly evolution of renewable energy generation in Spain from August 2013 to August 2025. It includes detailed information by energy source, reporting the total electricity sold (GWh), installed capacity (MW), and the cumulative number of active installations nationwide. This structure covers both production and infrastructure growth across renewable technologies.
#### **Data source**
The dataset used for this project was extracted from CNMC Data:
*“Monthly evolution of electricity generated from renewable energy sources, cogeneration, and waste in Spain. Energy sold and installed capacity by technology.”*
 https://data.cnmc.es/energia/energia-electrica/renovables-cogeneracion-y-residuos/evolucion-mensual-de-energia-electrica

___

### **2. Descriptive Analysis**

#### **Data Extraction**

In [1]:
# import libraries
import math
import requests
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy import stats

In [2]:
# helper functions
def format_subplots(fig, subplot_titles):
    for i, ann in enumerate(fig.layout.annotations):
        ann.update(font=dict(family="Helvetica", size=16))
        ax = f"xaxis{i + 1}"
        if ax in fig.layout:
            left = fig.layout[ax].domain[0]
            ann.update(
                x=left,
                xanchor="left",
                align="left",
                text=f"<b>{subplot_titles[i]}</b>",
                yshift=5,
            )
        fig.update_layout(template="plotly_white", font=dict(family="Helvetica"))
        fig.update_xaxes(title_font=dict(family="Helvetica", size=14))
        fig.update_yaxes(title_font=dict(family="Helvetica", size=14))
        fig.update_layout(margin=dict(t=150))
def format_plot(fig, template, height, width, title, subtitle, font, title_font=16,legend=False,tick_size=14):
    fig.update_layout(
        template=template,
        showlegend=legend,
        height=height,
        width=width,
        title={
            "text": f"<b>{title}</b><br>"
            f"<span style='font-size:16px;font-weight:400;'>{subtitle}</span>"
        },
        title_font=dict(family=font, size=title_font),
        title_x=0.0,
        legend_font=dict(size=14)
    )
    fig.update_xaxes(tickfont=dict(family=font, size=tick_size))
    fig.update_yaxes(tickfont=dict(family=font, size=tick_size))
def dickey(name, serie):
    adf_test = adfuller(serie.dropna())
    if adf_test[1] > 0.05:
        print(f"{name}: The series is non-stationary")
    else:
        print(f"{name}: The series is stationary. Ready to model")
    print("ADF Statistic:", round(adf_test[0], 3))
    print(f"p-value: {round(adf_test[1], 3)}\n")
def plot_acf(
    data,
    lags,
    type="acf",
    alpha=0.05,
    bar_color="rgba(255, 106, 0, 1)",
    marker_color="rgba(188, 78, 0, 1)",
    ci_color="rgba(255, 106, 0, 0.1)",
    row=1,
    col=1,
):
    if type=='acf':
        acf_vals, conf = acf(data, nlags=lags, fft=False, alpha=alpha)
    elif type=='pacf':
        acf_vals, conf = pacf(data, nlags=lags, method="ywm", alpha=alpha)
    lags = np.arange(len(acf_vals))
    fig.add_bar(  # acf bars
        x=lags,
        y=acf_vals,
        width=0.2,
        marker=dict(color=bar_color, line=dict(width=0)),
        showlegend=False,
        row=row,
        col=col,
    )
    fig.add_scatter(  # acf markers
        x=lags,
        y=acf_vals,
        mode="markers",
        name="lag",
        marker=dict(symbol="circle", size=4.5, color=marker_color),
        showlegend=False,
        row=row,
        col=col,
    )
    fig.add_scatter(  # confidence interval lower
        x=lags,
        y=conf[:, 0] - acf_vals,
        mode="lines",
        line=dict(width=0),
        fill="tonexty",
        fillcolor=ci_color,
        showlegend=False,
        hoverinfo="skip",
        hovertemplate=None,
        row=row,
        col=col,
    )
    fig.add_scatter(  # confidence interval upper
        x=lags,
        y=conf[:, 1] - acf_vals,
        mode="lines",
        line=dict(width=0),
        fill="tonexty",
        fillcolor=ci_color,
        showlegend=False,
        hoverinfo="skip",
        hovertemplate=None,
        row=row,
        col=col,
    )

In [3]:
# colors
colors = [
    "#74d200", 
    "#00a6ff", 
    "#ff6a00"]
colors_marker = [
    "rgba(84, 152, 0, 1)",
    "rgba(0, 118, 181, 1)",
    "rgba(188, 78, 0, 1)",
]
colors_shade = [
    "rgba(116, 210, 0, 0.1)",
    "rgba(0, 166, 255, 0.1)",
    "rgba(255, 106, 0, 0.1)",
]
forecast_palette = {
    "series":   "#ff6a00", 
    "fitted":   "#ffaa00", 
    "forecast": "#6600ff"}
solar_colors = [
    "#ff6a00", 
    "#ea4a00",
    "#d93600", 
    "#A31600"]

In [4]:
# data extraction
base_url = "https://catalogodatos.cnmc.es/api/action/datastore_search"
resource_id = "5131df61-db49-46d3-8210-7950d184ba50"

params = {'resource_id':resource_id, 'limit':2000}
response = requests.get(base_url,params=params)
response.raise_for_status()
data = response.json()
records = data['result']['records']
df = pd.DataFrame(records)


____

#### **Data Exploration**
The dataset contains **145 monthly observations spanning August 2013 to August 2026**, with each observation broken down into nine renewable energy technologies: Biomass, Cogeneration, Wind, Hydropower, Waste, Solar PV, Solar Thermal, Waste Treatment, and Other Renewable Sources.
Each monthly entry reports three key metrics for every technology:
- **Electricity sold** (GWh)
- **Installed capacity** (MW)
- **Cumulative number of installations** nationwide

This structure results in a dataset with 1,305 total rows (145 months × 9 technologies).

A quick inspection shows that the dataset is complete, **containing no missing values across any variable**.
The variable types are consistent with expectations with the exception of 'date', which requires to be transformed for time series modeling.

Summary statistics illustrate the wide dispersion in renewable technologies in Spain.
- **Energy sold** ranges from near-zero values for emerging technologies to over 7,600 GWh for established sources.
- **Installed capacity** varies from a few megawatts to over 32,000 MW, showing significant infrastructure heterogeneity.
- **Number of installations** is especially skewed: technologies like Solar PV have tens of thousands of installations, while Hydropower or Solar Thermal have only a few dozen to a few hundred. This could be attributed to high differences in the cost per new installation across technologies.

In August 2025, **Solar PV** and **Wind** dominate across energies by total energy sold and capacity installed. This supports its selection as the focus of the time series modeling in later sections.

In [5]:
# summarize dataframe + overview + descriptive statistics
summary = pd.DataFrame(
    {"data type": df.dtypes,
     "non-null values": df.notna().sum(),
     "nulls": df.isna().sum(),
     "unique values": df.nunique(),
    })
display('Dataset structure',summary)

df_mo= df.groupby('fecha')[['energia_vendida','potencia_instalada','n_instalaciones']].sum()
print("\n")
display("Dataset overview August 2013", df_mo.iloc[0].to_frame().T)
display(df.head(9))
print("\n")
display("Dataset overview August 2025", df_mo.iloc[-1].to_frame().T)
display(df.tail(9))
print("\n")
display('Summary statistics',df.describe())

'Dataset structure'

,data type,non-null values,nulls,unique values
_id,int64,1305,0,1305
fecha,object,1305,0,145
tecnologia,object,1305,0,9
energia_vendida,float64,1305,0,1305
potencia_instalada,float64,1305,0,333
n_instalaciones,float64,1305,0,291


'Dataset overview August 2013'

,energia_vendida,potencia_instalada,n_instalaciones
2013-08,7861.3921,39341.44058,64473.0


,_id,fecha,tecnologia,energia_vendida,potencia_instalada,n_instalaciones
0,1,2013-08,Biomasa,352.62557,752.46541,199.0
1,2,2013-08,Cogeneración,1492.50076,5489.68870,902.0
2,3,2013-08,Eólica,3445.08717,22965.53535,1351.0
3,4,2013-08,Hidráulica,396.70938,2040.01578,1079.0
4,5,2013-08,Otras tecn. renovables,0.00884,4.79600,2.0
5,6,2013-08,Residuos,269.20350,667.52100,33.0
6,7,2013-08,Solar FV,893.49215,4644.99934,60809.0
7,8,2013-08,Solar TE,659.43489,2149.62000,47.0
8,9,2013-08,Trat.residuos,352.32984,626.79900,51.0


'Dataset overview August 2025'

,energia_vendida,potencia_instalada,n_instalaciones
2025-08,11366.74987,75360.80613,66805.0


,_id,fecha,tecnologia,energia_vendida,potencia_instalada,n_instalaciones
1296,1297,2025-08,Biomasa,345.32034,1020.35041,234.0
1297,1298,2025-08,Cogeneración,952.66827,5263.25970,872.0
1298,1299,2025-08,Eólica,3699.35512,32312.47785,1748.0
1299,1300,2025-08,Hidráulica,215.63592,2147.61318,1100.0
1300,1301,2025-08,Otras tecn. renovables,0.01131,4.79600,2.0
1301,1302,2025-08,Residuos,165.24703,688.24500,34.0
1302,1303,2025-08,Solar FV,5247.19452,31010.56499,62715.0
1303,1304,2025-08,Solar TE,487.04263,2299.42000,50.0
1304,1305,2025-08,Trat.residuos,254.27473,614.07900,50.0


'Summary statistics'

,_id,energia_vendida,potencia_instalada,n_instalaciones
count,1305.000000,1305.000000,1305.000000,1305.000000
mean,653.000000,1050.098890,5491.096458,7264.453640
std,376.865361,1489.048796,8550.150309,19200.687388
min,1.000000,0.000560,4.796000,2.000000
25%,327.000000,211.188570,662.665000,50.000000
50%,653.000000,360.123470,2144.436180,227.000000
75%,979.000000,1114.821100,5286.463700,1100.000000
max,1305.000000,7667.687560,32312.477850,62715.000000


____

#### **Data Cleaning**
**1. Columns renaming and removal** 

The original dataset included column names in Spanish and an index-like `_id` field with no analytical purpose.
We renamed all variables to concise, English names and removed the unused identifier:
- fecha → date
- tecnologia → technology
- energia_vendida → energy_sold
- potencia_instalada → capacity_installed
- n_instalaciones → n_installed
- dropped _id

**2. Harmonizing technology names**

Energy technologies appeared in Spanish, with accents and abbreviations.
We replaced all names with standardized English equivalents:
- “Eólica” → “Wind”
- “Hidráulica” → “Hydropower”
- “Solar FV” → “Solar PV”
- “Trat.residuos” → “Waste treatment”...

This step is essential because these names become categorical identifiers for filtering, grouping, and selecting specific technologies (Solar PV for modeling later).

**3. Correcting data types**

The date column was parsed into a proper datetime format, enabling time-based indexing and filtering. Numeric variables (energy_sold, capacity_installed, n_installed) were explicitly converted to numeric types, ensuring clean arithmetic operations and preventing type-related errors during visualization and modeling.

**4. Filtering and sorting**

The project focuses on the last decade of renewable energy evolution. Therefore, **observations prior to 2015 were removed.** The dataset was sorted by date and technology to maintain a consistent monthly order.


In [6]:
# rename & drop columns
new_column_names = {
    "_id": "id",
    "fecha": "date",
    "tecnologia": "technology",
    "energia_vendida": "energy_sold",
    "potencia_instalada": "capacity_installed",
    "n_instalaciones": "n_installed",
}
df = df.rename(columns=new_column_names)
df = df.drop(columns=["id"])

# rename technologies
new_technologies_names = {
    "Biomasa": "Biomass",
    "Cogeneración": "Cogeneration",
    "Eólica": "Wind",
    "Hidráulica": "Hydropower",
    "Otras tecn. renovables": "Other renewables",
    "Residuos": "Waste",
    "Solar FV": "Solar PV",
    "Solar TE": "Solar Thermal",
    "Trat.residuos": "Waste treatment",
}
df["technology"] = df["technology"].replace(new_technologies_names)

# correct data types
df["date"] = pd.to_datetime(df["date"],format='%Y-%m')
numeric_columns = ["energy_sold", "capacity_installed", "n_installed"]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col],errors='coerce')

# filter dates
df = df[ (df['date'].dt.year >= 2015) ]

# sort values: date and technology
df = df.sort_values(["date", "technology"]).reset_index(drop=True)
df.head(len(df['technology'].unique()))

,date,technology,energy_sold,capacity_installed,n_installed
0,2015-01-01,Biomass,341.66799,791.39441,206.0
1,2015-01-01,Cogeneration,1941.26009,5502.37120,906.0
2,2015-01-01,Hydropower,533.68290,2092.35958,1086.0
3,2015-01-01,Other renewables,0.03756,4.79600,2.0
4,2015-01-01,Solar PV,517.15575,4654.12462,60946.0
5,2015-01-01,Solar Thermal,176.94175,2299.42750,51.0
6,2015-01-01,Waste,248.01919,667.52100,33.0
7,2015-01-01,Waste treatment,131.87249,626.79900,51.0
8,2015-01-01,Wind,4943.03433,22999.90585,1357.0


___

#### **Metrics Evaluation**

In [7]:
# time series metric
series_energy = df.groupby("date")['energy_sold'].sum()
series_capacity = df.groupby("date")["capacity_installed"].sum()
series_installed = df.groupby("date")["n_installed"].sum()

In [8]:
# time series details
series = {
    "Energy Sold (GWh)": (series_energy,"energy_sold", "GHw"),
    "Capacity Installed (MW)": (series_capacity,"capacity_installed", "MW"),
    "Installations Number": (series_installed,"n_installed", "Installations"),
}

# plots
subplot_titles = tuple(series.keys())
fig = make_subplots(
    rows=1, cols=3,
    shared_xaxes=False, shared_yaxes=False,
    subplot_titles=subplot_titles,
)
for col, (name, (s, metric, unit)) in enumerate(series.items(), start=1):
    fig.add_scatter(x=s.index, y=s, mode="lines", name=name, row=1, col=col)
    fig.update_yaxes(title_text=unit, row=1, col=col)

# format plot and subplots
format_subplots(fig, subplot_titles)
format_plot(
    fig,
    template="plotly_white",
    height=500,
    width=1350,
    title="Metrics Evaluation: Energy Sold (GWh), Capacity Installed (MV) and Installations Number in Spain (2015-2025)",
    subtitle="Energy Sold stands out, displaying strong trend and seasonal components",
    font='Helvetica',
    title_font=18
)
fig.show()

**Why we model Energy Sold instead of Installed Capacity or Number of Installations**

Among the 3 available metrics: energy sold, installed capacity, and number of installations, **energy sold is the most suitable target for time series forecasting**. Unlike the other two metrics, energy sold displays a pronounced seasonal pattern.

Installed capacity and number of installations, by contrast, behave more like cumulative growth curves. These metrics increase almost monotonically over time, with long periods of flat behavior followed by sudden jumps driven by infrastructure expansions rather than underlying temporal dynamics. As a result, they are poor candidates for ARIMA/SARIMA models, which assume recurring patterns and meaningful autocorrelation.

Energy sold, however, reflects the actual operational output of Solar PV systems and incorporates the effects of weather, seasonality, infrastructure expansion, and usage patterns. This makes it both rich in time-dependent structure and highly relevant for forecasting the evolution of solar generation in Spain.

___

#### **Technologies Evaluation**

In [9]:
# time series by technology type
technologies = df["technology"].unique()
series = {}
for tec in technologies:
    tec_df = df[df["technology"] == tec]
    series[tec] = tec_df.groupby("date")["energy_sold"].sum()


In [10]:
# plot technologies
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.1,
)
for idx, (name, s) in enumerate(series.items()):
    fig.add_scatter(x=s.index, y=s, name=name, row=1, col=1)
# format plot and subplots
format_plot(
    fig,
    "plotly_white",
    600,
    1350,
    "Energy Sold by Technology Type in Spain (2015-2025)",
    "Solar PV overtakes Wind as the main renewable source in Spain in terms of Energy Sold",
    "Helvetica",
    20,
)
fig.update_layout(
    showlegend=True, 
    legend=dict(orientation="h", y=1.05, x=0),
    font=dict(family='Helvetica')
) 
fig.show()


# subplots technologies
n = len(series)
cols = 3
rows = math.ceil(n / cols)
subplot_titles = tuple(series.keys())
fig2 = make_subplots(
    rows=rows,
    cols=cols,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1,
)
for idx, (name, s) in enumerate(series.items()):
    row = idx // cols + 1
    col = idx % cols + 1
    fig2.add_scatter(x=s.index, y=s, name=name, row=row, col=col)
# format plot and subplots
format_subplots(fig2,subplot_titles)
format_plot(
    fig2,
    "plotly_white",
    900,
    1350,
    "Energy Sold by Technology Type in Spain (2015-2025)",
    "Wind, Hydropower and Solar Thermal display additive seasonality. Solar Photovoltaic displays clear multiplicative seasonality",
    'Helvetica',
    18
)
fig2.show()


# prospective time series
pros = ["Total", "Wind", "Solar PV"]
series_map = {
    pros[0]: df.groupby("date")["energy_sold"].sum(),
    pros[1]: series[pros[1]],
    pros[2]: series[pros[2]],
}
# plots pros
subplot_titles = tuple(series_map.keys())
fig3 = make_subplots(
    rows=1,
    cols=3,
    shared_xaxes=False,
    shared_yaxes=True,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1,
)
for col, (name, s) in enumerate(series_map.items(), start=1):
    fig3.add_scatter(
        x=s.index, y=s, name=name, line=dict(color=colors[col - 1]), row=1, col=col
    )
# format plot and subplots
format_subplots(fig3, subplot_titles)
format_plot(
    fig3,
    "plotly_white",
    600,
    1350,
    "Prospective Time Series</b>",
    "Total Energy, Wind, and Solar PV",
    "Helvetica",
    18,
)
fig3.show()



**Why we selected Total Energy Sold, Wind Energy Sold, and Solar PV Energy Sold as modeling candidates**

After examining the energy sold by each renewable technology in Spain, three series stand out as the most suitable and informative candidates for forecasting: total energy sold, wind, and solar PV. These choices are motivated by both their statistical properties and their relevance within Spain’s renewable energy mix.

1. **Total:** This series aggregates the contribution of all renewable sources and therefore provides the broadest view of Spain’s renewable generation profile. It exhibits a clear upward trend and stable seasonal pattern, making it well-suited for classical time-series modeling. Forecasting total renewable output is also highly relevant from an energy policy and grid-planning perspective.

2. **Wind:** Wind remains one of the largest contributors to renewable generation in Spain. The series shows strong additive seasonality with regular winter peaks and summer troughs. Its well-structured oscillations make it an ideal candidate for SARIMA-type models, and modeling wind output offers operational insight into one of Spain’s most mature renewable technologies.

3. **Solar PV:** One of the fastest-growing renewable sources, and its series displays a distinctive combination of **multiplicative seasonality** and rapid long-term growth. This creates a rich and interesting modeling challenge: the seasonality grows alongside the trend, and structural changes in the industry are reflected in the series. Forecasting Solar PV is especially valuable because it represents the future trajectory of Spain’s clean-energy expansion.

___

#### **Decomposition** 

In [11]:
# series and models
series_map_d = {
    pros[0]: (df.groupby("date")["energy_sold"].sum(), "additive"),
    pros[1]: (series[pros[1]], "additive"),
    pros[2]: (series[pros[2]], "multiplicative"),
}

# plots
subplot_titles = [
    f"{pros[0]}",
    f"{pros[1]}",
    f"{pros[2]}",
    f"{pros[0]} (Trend)",
    f"{pros[1]} (Trend)",
    f"{pros[2]} (Trend)",
    f"{pros[0]} (Seasonal)",
    f"{pros[1]} (Seasonal)",
    f"{pros[2]} (Seasonal)",
    f"{pros[0]} (Residuals)",
    f"{pros[1]} (Residuals)",
    f"{pros[2]} (Residuals)",
]
fig = make_subplots(
    rows=4,
    cols=3,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1
)
for col, (name, (s,model)) in enumerate(series_map_d.items(), start=1):
    color = colors[col - 1]
    color_mark = colors_marker[col - 1]
    color_shade = colors_shade[col - 1]

    # decomposition
    decomp = seasonal_decompose(s, model = model, period=12)
    trend = decomp.trend
    seasonal = decomp.seasonal
    residual = decomp.resid

    # plot
    fig.add_scatter(x=s.index, y=s, mode="lines", name=name, line=dict(color=color), row=1, col=col)
    fig.add_scatter(x= s.index, y=trend, mode='lines', name= f"{pros[col-1]}", line=dict(color=colors[col-1]), row=2, col=col)  # trend
    fig.add_scatter(x= s.index, y=seasonal, mode='lines', name= f"{pros[col-1]}", line=dict(color=colors[col-1]), row=3, col=col)  # season
    fig.add_scatter(x= s.index, y=residual, mode='lines', name= f"{pros[col-1]}", line=dict(color=colors[col-1]), row=4, col=col)  # residuals

# format plot and subplots
format_subplots(fig,subplot_titles)
format_plot(
    fig,
    "plotly_white",
    900,
    1350,
    "Time Series Decomposition",
    "Additive model used for Total and Wind. Multiplicative model used for Solar PV",
    'Helvetica',
    18
)    
fig.show()

**Why we decomposed each time series and how to select the correct model**

To better understand the structure of our three candidate time series (Total Energy Sold, Wind Energy Sold, and Solar PV Energy Sold) we decomposed each into trend, seasonal, and residual components. This step helps identify the underlying patterns that guide our model selection.

**1. Total Energy Sold — Additive model**

This series display a smooth upward trend, reflecting steady growth in renewable generation overall. The seasonal swings remain roughly constant in magnitude over time. As the trend grows, the amplitude of the seasonal peaks does not expand proportionally. This behavior matches the assumption of additive structure: trend and fixed-size seasonal deviations.


**2. Wind Energy Sold — Additive model**

This series displays a clear seasonal cycle with winter peaks and summer troughs. Wind generation shows strong, regular yearly seasonality, but the amplitude of these seasonal peaks is largely stable over time. There is no multiplicative effect. The seasonal fluctuations do not scale with the level of the trend.


**3. Solar PV Energy Sold — Multiplicative model**

This series exhibits rapid growth over time; the level of the series increases several-fold. The seasonal amplitude increases with the trend—seasonal peaks become larger as total production increases. Seasonal fluctuations are proportional to the level of the series, a feature of multiplicative seasonality.

___

#### **Autocorrelation Evaluation**


In [12]:
# plots
subplot_titles = [pros[0], pros[1], pros[2],
                  f"ACF of {pros[0]}", f"ACF of {pros[1]}", f"ACF of {pros[2]}",
                  f"PACF of {pros[0]}", f"PACF of {pros[1]}", f"PACF of {pros[2]}"]
fig = make_subplots(
    rows=3,
    cols=3,
    shared_xaxes=False,
    shared_yaxes=True,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1,
)
max_lags = 36
for col, (name, s) in enumerate(series_map.items(), start=1):
    color = colors[col-1]
    color_mark = colors_marker[col - 1]
    color_shade = colors_shade[col - 1]

    fig.add_scatter(x=s.index, y=s, mode="lines", name=name,line=dict(color=color), row=1, col=col)
    plot_acf(s, max_lags, "acf", 0.05, color, color_mark, color_shade, row=2, col=col)
    plot_acf(s,max_lags,'pacf',0.05,color,color_mark,color_shade,row=3,col=col)
# format plot and subplots
format_subplots(fig,subplot_titles)
format_plot(
    fig,
    "plotly_white",
    900,
    1350,
    "Autocorrelation Evaluation: </b>ACF and PACF",
    "Observable non-stationary series with clear trend and seasonal components",
    'Helvetica',
    18
)
fig.show()


**How we analyzed the temporal structure of our time series through ACF and PACF**

The Autocorrelation Functions (ACF) and Partial Autocorrelation Functions (PACF) were key to understand dependence on prior periods, identify non-stationarity, understand their seasonal patterns. Below is our interpretation for each series:

**1. Total Energy Sold**

- **ACF:** Slow decay across lags, a strong indication of non-stationarity with pronounced seasonal spikes every 12 periods. 

- **PACF:** Strong early lags 1 and 2 indicate direct short-term statistical persistance and signal a stochastic trend component. Additionally, some persistance at seasonal lags 10 and 20 reinforces yearly structure.

- **Interpretation:** Total Energy incorporates both stochastic trend and seasonal components. This temporal structure calls for first and seasonal differencing. Without a stationary behavior in the series we cannot conclude about the order of autorregressive or moving-average components.

**2. Wind Energy Sold**

- **ACF:** Displays slow decaying oscillation pattern, confirming a strong seasonality with peaks every 12 months. Oscilation can be attributed to seasonal patterns influencing wind energy generation.

- **PACF:** Pronounced decay after lag 1 with moderate decaying oscillation roughly every seasonal period (12 months). This confirms stochastic seasonal component mainly from the first seasonal period.

- **Interpretation:** This series displays the need for first and seasonal differencing due to its strong non-stationarity and seasonality component. In addition, strong and moderate oscillation patterns can be attributed to weather patterns influencing wind energy production.


**3. Solar PV Energy Sold**
- **ACF:** Slow exponential-like decay displays clear non-stationarity. Additionally, we can evidence clear seasonal spikes appear every 12 periods.

- **PACF:** Strong and direct persistance from early lags indicates the need for diffencing. Following lags display small oscillating pattern with peaks every seasonal period, calling for the need of seasonal differencing.

- **Interpretation:** This series displays the need for logarithmic transformation due to its exponential growth. In addition, first and seasonal differencing might be needed to capture the temporal structure of this series.

___

#### **Stationarity Diagnostic**

In [13]:
# Augmented Dickey-Fuller test
for name, s in series_map.items():
    dickey(name, s)

Total: The series is non-stationary
ADF Statistic: -0.58
p-value: 0.875

Wind: The series is non-stationary
ADF Statistic: -1.15
p-value: 0.695

Solar PV: The series is non-stationary
ADF Statistic: 0.666
p-value: 0.989



**How the Dickey–Fuller Test helped us assess non-stationarity**

We applied the Augmented Dickey–Fuller (ADF) test to the Total Energy Sold, Wind Energy Sold, and Solar PV Energy Sold series to assess whether each series is stationary. The test evaluates the null hypothesis that a unit root is present, meaning the series is non-stationary.

**1. Total Energy Sold:** 

The ADF statistic is –0.58 with a p-value of **0.875.**
Because the p-value is far above our 5% significance level, we fail to reject the null hypothesis. This confirms that the Total Energy Sold series behaves as a non-stationary process. The weak test statistic indicates limited evidence of mean reversion and supports the presence of trend or persistent growth patterns.

**2. Wind Energy Sold**

The ADF statistic is –1.15 with a p-value of **0.695.**
Again, the p-value is well above our threshold, so we fail to reject the null hypothesis. The Wind series is therefore non-stationary. Its test statistic is slightly more negative than the Total series, but still nowhere near the critical values required for stationarity. This suggests the series retains long-term structure, consistent with visible seasonality and long-run trend.

**3. Solar PV Energy Sold**

The ADF statistic is 0.666 with a p-value of **0.989.**
This is the strongest indication of non-stationarity among the three series: the test statistic is positive, and the p-value is extremely high. We clearly fail to reject the null hypothesis. Solar PV behaves as a highly non-stationary series, likely driven by strong trend growth and evolving seasonal dynamics typical of rapidly expanding renewable technologies.


___

### **3. Time Series Evaluation**

In [14]:
solar = series_map['Solar PV']

# plots
subplot_titles = (
    "Solar PV",
    "ACF of Solar PV",
    "PACF of Solar PV",
)
fig = make_subplots(
    rows=1, cols=3, shared_xaxes=False, shared_yaxes=False, subplot_titles=subplot_titles
)
fig.add_scatter(
    x=solar.index,
    y=solar,
    name="Solar",
    mode="lines",
    line=dict(color=colors[2]),
    showlegend=False,
    row=1,
    col=1,
)
plot_acf(solar,max_lags,'acf',0.05,row=1,col=2) #acf
plot_acf(solar,max_lags,'pacf',0.05,row=1,col=3) #pacf

# format plot and subplots
format_subplots(fig, subplot_titles)
format_plot(
    fig,
    "plotly_white",
    500,
    1350,
    "Solar Photovoltaic Energy Sold in Spain (2015-2025)</b>",
    "",
    "Helvetica",
    18,
)
fig.show()


**Why we chose Solar Photovoltaic Energy Sold for our model**

Solar Photovoltaic emerged as the strongest candidate for modeling because its behavior shows the richest and most well-defined time-series structure. The series displays a clear seasonal pattern, a pronounced upward trend, and strong autocorrelation, all of which provide meaningful temporal signals for a statistical model to capture. Compared to Total and Wind energy sold, Solar PV has more distinct and expanding seasonal amplitudes, making it highly suitable for stationarity transformations, and model-based forecasting.

___

#### **Transformation and Differentiation**

In [15]:
# transformations
solar = series_map["Solar PV"]
solar_log = np.log(solar)
solar_log_diff = solar_log.diff(1).dropna()
solar_log_diff_seas = solar_log_diff.diff(12).dropna()

#  plots
fig = make_subplots(
    rows=4, 
    cols=1, 
    shared_xaxes=True, 
    shared_yaxes=False,
    subplot_titles=subplot_titles,
    vertical_spacing=0.1)
subplot_titles = (
    "Solar",
    "Solar (log)",
    "Solar (log) diff(1)",
    "Solar (log) diff(1) + diff(12)"
)

fig.add_scatter( #solar
    x=solar.index, 
    y=solar, 
    mode='lines',
    line=dict(color=solar_colors[0]),
    name=subplot_titles[0],
    showlegend=False, 
    row=1,col=1)
fig.add_scatter( #solar (log)
    x=solar_log.index, 
    y=solar_log, 
    mode='lines',
    line=dict(color=solar_colors[1]),
    name='log',
    showlegend=False, 
    row=2, col=1)
fig.add_scatter( #solar (log) diff1
    x=solar_log_diff.index, 
    y=solar_log_diff, 
    mode='lines',
    line=dict(color=solar_colors[2]),
    name='first diff',
    showlegend=False, 
    row=3,col=1)
fig.add_scatter( #solar (log) diff1 diff12
    x=solar_log_diff_seas.index, 
    y=solar_log_diff_seas, 
    mode='lines',
    line=dict(color=solar_colors[3]),
    name='seasonal',
    showlegend=False, 
    row=4,col=1)

# format plot and subplots
format_subplots(fig,subplot_titles)
format_plot(
    fig,
    "plotly_white",
    900,
    1350,
    "Solar PV Transformation & Differencing",
    "Logarithmic transformation, non-seasonal and seasonal differentiation",
    'Helvetica',
    18
)
fig.show()

**Why we applied different transformations and differencings to our original series**

We applied a series of transformations to the Solar PV time series to stabilize its structure and make it suitable for ARIMA-type modeling. 

- First, a logarithmic transformation was used to **reduce the increasing variance** visible in the raw data and to convert the multiplicative seasonal pattern into an approximately additive one. 

- After stabilizing variance, we applied first differencing to **remove the long-term upward trend**, leaving a series that fluctuates around a constant mean. 

- Finally, we introduced seasonal differencing at the annual frequency to **eliminate the recurring seasonal cycle**. Together, these steps remove trend and seasonality while stabilizing variability, producing a stationary series that meets the assumptions required for effective model estimation.

___

#### **Transformations Stationarity Diagnostic** 

In [16]:
trans = {
    'Solar (log)': solar_log,
    'Solar (log) diff(1)': solar_log_diff,
    'Solar (log) diff(1) diff(12)': solar_log_diff_seas
}
for name, series in trans.items():
    dickey(name,series)

Solar (log): The series is non-stationary
ADF Statistic: 0.188
p-value: 0.972

Solar (log) diff(1): The series is non-stationary
ADF Statistic: -2.55
p-value: 0.104

Solar (log) diff(1) diff(12): The series is stationary. Ready to model
ADF Statistic: -4.039
p-value: 0.001



**Re-assessing stationarity for our transformed series**

We re-evaluated stationarity after each transformation step to confirm whether the series was suitable for modeling. 

- The log-transformed Solar PV series remained non-stationary, as indicated by an ADF statistic of 0.188 and a high p-value (0.972), showing that stabilizing variance alone did not eliminate the underlying trend or seasonal structure. 

- Applying first differencing brought the series closer to stationarity, but the ADF statistic (–2.55) and p-value (0.104) still did not meet the threshold to reject the unit-root hypothesis, meaning trend or seasonal persistence was still present. 

- Once we added seasonal differencing at lag 12, the ADF statistic dropped to –4.039 with a p-value of 0.001, allowing us to reject the null hypothesis and conclude that the transformed series **is now stationary.** 

This confirms that a combination of log transformation, non-seasonal differencing, and seasonal differencing is required to prepare the Solar PV series for ARIMA modeling.

___

#### **Transformation Autocorrelation Evaluation**

In [17]:
solars = solar_log_diff_seas

# plots
subplot_titles = (
    "Solar (transformed)",
    "ACF of Solar (transformed)",
    "PACF of Solar (transformed)",
)
fig = make_subplots(
    rows=1, cols=3, shared_xaxes=False, shared_yaxes=True, subplot_titles=subplot_titles
)
fig.add_scatter(
    x=solars.index, 
    y=solars, 
    name='Solar',
    mode= 'lines',
    line=dict(color=solar_colors[0]),
    showlegend=False,
    row=1,col=1)
plot_acf(solars, max_lags, 'acf', 0.05, row=1, col=2) #ACF solar (transformed)
plot_acf(solars,max_lags,'pacf',0.05,row=1,col=3)  #PACF solar (transformed)

# format plot and subplots
format_subplots(fig,subplot_titles)
format_plot(
    fig,
    "plotly_white",
    500,
    1350,
    "Transformation Autocorrelation Evaluation</b>",
    "ACF and PACF indicate the need for a model with low-order MA term for seasonal and non-seasonal components",
    'Helvetica',
    18
)
fig.show()


**How we evaluated ACF and PACF of transformed series to inform model evaluation**

The ACF and PACF of the fully transformed Solar PV series show the characteristic pattern of a process that requires low-order MA terms and potentially AR terms. 

- The ACF plot has a strong lag 1 spike, followed a considerable decay. In addition we evidence persistent spikes around the first seasonal period. 

- The PACF plot displays a strong spike at lag 1 followed by a gradual decay rather than a clean cutoff. This behavior calls for a low-order MA term in the series modeling.

This behavior indicates that a seasonal MA structure is might be sufficient but AR terms might still be relevant for our model. The remaining autocorrelations, although small, persist across several lags, suggesting short-range dependence that must be captured by non-seasonal ARMA components. 

Additionally, the presence of meaningful correlations at multiples of the seasonal lag reflects the need to incorporate seasonal AR or MA terms in the model. Overall, the ACF and PACF confirm that the transformed series possibly contain both non-seasonal and seasonal dynamics, guiding us toward a SARIMA specification with low-order components in both the MA and potentially AR parts.

___

### **4. Model Evaluation**

In [18]:
solar_log = solar_log.asfreq('MS')
# SARIMA(p, d, q)(P, D, Q)[s]
candidates = {
    "SARIMA(1, 1, 1)(1, 1, 1)[12]": ((1, 1, 1), (1, 1, 1, 12)),
    "SARIMA(1, 1, 1)(0, 1, 1)[12]": ((1, 1, 1), (0, 1, 1, 12)),
    "SARIMA(1, 1, 1)(1, 1, 0)[12]": ((1, 1, 1), (1, 1, 0, 12)),
    "SARIMA(0, 1, 1)(0, 1, 1)[12]": ((0, 1, 1), (0, 1, 1, 12)),
}
results = []
best_bic = 0
for name, (order, seasonal_order) in candidates.items():
    model = SARIMAX(solar_log, order=order, seasonal_order=seasonal_order, enforce_invertibility=False, enforce_stationarity=False)
    fit = model.fit(disp=False)
    results.append([name, fit.aic, fit.bic])
    if fit.bic < best_bic:
        best_bic = fit.bic
        best_fit = fit
results_table = pd.DataFrame(results, columns=['Model','AIC','BIC']).sort_values('BIC').reset_index(drop=True)
display(results_table)
print("\n")
print(best_fit.summary())

/Users/nicolaswilches/MBDS/term1/pyda/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



,Model,AIC,BIC
0,"SARIMA(0, 1, 1)(0, 1, 1)[12]",-94.875909,-87.030547
1,"SARIMA(1, 1, 1)(0, 1, 1)[12]",-93.149734,-82.689252
2,"SARIMA(1, 1, 1)(1, 1, 1)[12]",-88.314253,-75.238651
3,"SARIMA(1, 1, 1)(1, 1, 0)[12]",-79.597521,-69.097630




                                     SARIMAX Results                                      
Dep. Variable:                        energy_sold   No. Observations:                  128
Model:             SARIMAX(0, 1, 1)x(0, 1, 1, 12)   Log Likelihood                  50.438
Date:                            Tue, 09 Dec 2025   AIC                            -94.876
Time:                                    23:18:52   BIC                            -87.031
Sample:                                01-01-2015   HQIC                           -91.700
                                     - 08-01-2025                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.6214      0.073     -8.474      0.000      -0.765      -0.478
ma.S.L12      -0.8474      0.153 


**Model Evaluation Summary for SARIMA(0,1,1)(0,1,1)[12]**

After testing multiple SARIMA configurations, this model emerged as the best-performing specification based on information criteria and diagnostic validation. This model integrates non-seasonal and seasonal MA components and both non-seasonal and seasonal differencing. This parameter configuration captures the short-term dynamics and the strong annual seasonality present in the Solar PV series.

**1. Model Selection Performance**

- This model achieved the lowest **AIC (–94.876)** and lowest **BIC (–87.031)** among the candidate models tested. Lower values in these metrics indicate a better trade-off between model fit and complexity. 

- The competing models, which added low-order non-seasonal seasonal AR terms, delivered worse AIC and BIC scores, suggesting that **AR parameters did not improve the model sufficiently to justify their inclusion.**

**2. Parameter Interpretation and Significance**

The parameter estimates reveal:

- **Non-seasonal MA(1):** The coefficient (–0.62) is strongly significant (p < 0.001), showing the series exhibits short-term shock persistence consistent with the strong ACF spike at lag 1.

- **Seasonal MA(1):** The coefficient at lag 12 (–0.85) is also highly significant (p < 0.001), confirming that annual seasonal shocks remain influential even after seasonal differencing.

- **Residual variance:** Estimated at 0.0199, this value is stable and statistically significant.

**3. Residual Diagnostics**

Residual analysis again verifies that the model is well specified:

- **Ljung–Box test (lag 1):** The high p-value (0.62) indicates no remaining autocorrelation, meaning the SARIMA model has captured the dependence structure adequately.

- **Heteroskedasticity test:** With p = 0.09, there is no significant evidence of changing variance, showing that the log transformation and differencing stabilized volatility successfully.

- **Normality (Jarque–Bera):** The JB test indicates some departure from normality (p < 0.01), with mild tail heaviness—common in energy-generation data—but not problematic for SARIMA-based forecasting.

- **Skewness and kurtosis:** Residual skewness is minimal (–0.22), and kurtosis (4.67) reflects modest heavy tails, again typical for real-world time series.

**4. Conclusion**

The SARIMA(0,1,1)(0,1,1)[12] model provides the best balance between fit quality and parsimony. It effectively models both the short-term dynamics and the pronounced yearly seasonality of the Solar PV series. The significant MA components highlight the dominant role of short-lived and seasonal shocks, while the residual diagnostics confirm a well-specified and reliable model suitable for forecasting.

___

#### **Model Fitting**

In [19]:
# model and residuals
solar_model = SARIMAX(solar_log,order=(0,1,1),seasonal_order=(0,1,1,12),
                      enforce_stationarity=False,enforce_invertibility=False)
fit = solar_model.fit(disp=False)

# drop initial residuals
resid = fit.resid.dropna()
resid = resid[13:]

#### **Residuals diagnostics**

In [20]:
# residuals plots
subplot_titles = (
    "Residuals",
    "ACF of Residuals",
    "ACF of Residuals Squared",
    "QQ Plot of Residuals",
)
fig = make_subplots(
    rows=2,
    cols=2,
    shared_xaxes=False,
    shared_yaxes=False,
    subplot_titles=subplot_titles,
)
fig.add_scatter(# solar (stationary)
    x=resid.index, 
    y=resid, 
    name="Residuals", 
    mode="lines", 
    line=dict(color=colors[2]), 
    showlegend=False, 
    row=1, 
    col=1)
plot_acf(resid,  max_lags, 'acf', 0.05, row=1, col=2) #residuals
plot_acf(resid**2, max_lags, 'acf', 0.05, row=2, col=1)  # ACF residuals squared

# QQ plot
(osm, osr), (slope, intercept, r) = stats.probplot(resid, dist="norm", fit=True, rvalue=True)
line = slope * np.array(osm) + intercept
fig.add_trace(go.Scatter( # points
    x=osm, 
    y=osr,
    mode="markers",
    marker=dict(symbol='circle',size=7,opacity=0.5),
    name=f"r = {r:.3f}"), 
    row=2, 
    col=2)
fig.add_trace(go.Scatter( #line
    x=osm, 
    y=line, 
    mode="lines", 
    name="Reference", 
    line=dict(color=solar_colors[2])), 
    row=2, col=2)
fig.update_xaxes(
    title="Theoretical Quantiles",
    title_font=dict(family="Helvetica", size=14), 
    row=2, 
    col=2)
fig.update_yaxes(
    title="Sample Quantiles",
    range=[-0.5, 0.5],title_font=dict(family="Helvetica", size=14), 
    row=2, 
    col=2)

# format subplots
format_subplots(fig,subplot_titles)
format_plot(
    fig,
    "plotly_white",
    900,
    1350,
    "Residuals Diagnostic",
    "Residuals are uncorrelated, homoscedastic and are approximately normally distributed",
    'Helvetica',
    18
)
fig.show()


**Residual Diagnostics Interpretation**

The residual diagnostics confirm that the SARIMA(0,1,1)(0,1,1)[12] model provides a well-specified fit for the transformed Solar PV series. The residual time series plot shows fluctuations centered around zero with no visible trend or seasonal pattern, indicating that the model has successfully removed systematic structure from the data.

The **ACF of the residuals** displays no significant autocorrelations beyond lag 0, with all spikes remaining well within the confidence bounds. This confirms that no meaningful temporal dependence remains. The model has therefore captured the short-term and seasonal correlation structure appropriately, leaving residuals consistent with white noise.

Similarly, the **ACF of squared residuals** contains no significant spikes, suggesting the absence of volatility clustering or time-varying variance. This supports the conclusion that the log transformation and differencing were effective in stabilizing variance, and that no ARCH/GARCH components are required.

The **QQ plot** shows residuals lying close to the 45-degree reference line, with mild deviations in the tails. This indicates approximate normality with slightly heavier tails—a common and acceptable pattern in real-world energy generation data. Importantly, these deviations do not compromise the model’s adequacy, given that SARIMA estimation does not strictly require perfectly normal residuals.

Overall, the diagnostics show that the residuals are uncorrelated, homoscedastic, and approximately normally distributed. This confirms that the SARIMA(0,1,1)(0,1,1)[12] model captures the underlying structure of the Solar PV series effectively and is suitable for reliable forecasting.

___

### **5. Forecasts**

In [21]:
# in-sample (fitted)
in_sample = fit.get_prediction(start=solar_log.index[0], end=solar_log.index[-1])
in_sample_log = in_sample.predicted_mean

# out-of-sample (forecasts)
n = 24
forecast = fit.get_forecast(steps=n)
forecast_log = forecast.predicted_mean
forecast_log_ci = forecast.conf_int()

# back-transform
solar_fitted = np.exp(in_sample_log)
solar_forecast = np.exp(forecast_log)

# confidence interval
lower, upper = forecast_log_ci.columns
forecast_lower = np.exp(forecast_log_ci[lower])
forecast_upper = np.exp(forecast_log_ci[upper])

# subplots
subplot_titles = ""
fig = make_subplots(
    rows=1, 
    cols=1, 
    shared_xaxes=True, 
    shared_yaxes=True, 
    subplot_titles=subplot_titles)

# plots
fig.add_scatter( #observed
    x=solar.index, 
    y=solar,
    name='Solar',
    mode='lines',
    line=dict(color=forecast_palette['series'],width=3),
    opacity=0.8)
fig.add_scatter( #fitted
    x=solar_fitted.index, 
    y=solar_fitted,
    name='Fitted',
    mode='lines',
    line=dict(color=forecast_palette['fitted'],dash='dot',width=3),
    opacity=0.8)
fig.add_scatter(  # forecast
    x=solar_forecast.index,
    y=solar_forecast,
    name="Forecast",
    mode="lines",
    line=dict(color=forecast_palette["forecast"], dash="dot",width=3),
    opacity=1,
)
fig.add_trace(go.Scatter( #confidence interval
    x=forecast_upper.index.tolist() + forecast_lower.index[::-1].tolist(),
    y=forecast_upper.tolist() + forecast_lower[::-1].tolist(),
    fill="toself",
    fillcolor="rgba(102, 0, 255, 0.05)",
    line=dict(color="rgba(0,0,0,0)"),
    hoverinfo="skip",
    showlegend=True,
    name="95% CI"))
fig.add_vrect( #highlight outlier
    x0=pd.to_datetime(solar.index[11],format='%Y-%m-%d'), 
    x1=pd.to_datetime(solar.index[13],format='%Y-%m-%d'),
    fillcolor="gold", 
    opacity=0.2, 
    line_width=0, 
    layer="below")

# format plot and subplots
fig.update_layout(template="plotly_white", font=dict(family="Helvetica"))
format_subplots(fig,subplot_titles)
fig.update_xaxes(title_font=dict(size=16), tickfont=dict(size=14), row=1, col=1)
fig.update_yaxes(title_font=dict(size=16), tickfont=dict(size=14), row=1, col=1)
format_plot(
    fig,
    "plotly_white",
    600,
    1350,
    "Solar Photovoltaic vs. Fitted and Forecast (2015-2027)",
    "First seasonal period included. Observable outlier in period 12",
    'Helvetica',
    18
)
fig.show()


**Forecast Interpretation and Forecasting Insights**

The **SARIMA(0,1,1)(0,1,1)[12]** model produces fitted values that closely track the observed Solar PV generation across the entire historical period, successfully capturing both the yearly seasonal peaks and the long-term growth pattern in the series. The forecast for 2025–2027 continues this trajectory, projecting a sustained upward trend with the familiar seasonal oscillations that characterize solar production. The forecast intervals expand gradually as we extend into the future, reflecting the increasing uncertainty inherent to multi-step-ahead forecasting.

**Observed vs. Fitted and Forecast (Including the First Seasonal Period)**

In the first forecast visualization, the spike highlighted around period 12 reflects a mechanical artifact of SARIMA reconstruction rather than an error in the model. This point corresponds to the first seasonal period after differencing (the first backward seasonal lag), where the original level of the series must be reconstructed by adding back the differenced seasonal component. Because this step depends on an early historical data point that differs markedly from the long-run pattern, the reconstruction produces an isolated outlier. This outlier is not a result of model dynamics, nor does it signal poor model fit; it arises solely from how seasonal differencing reconstructs the initial fitted value.

In [22]:
# subplots
trim = 14
fig.update_xaxes(range=[solar.index[trim], solar_forecast.index[-1]])
fig.update_yaxes(range=[0, max(solar.max(), solar_forecast.max()) * 1.1])
format_plot(
    fig,
    "plotly_white",
    600,
    1350,
    "Solar Photovoltaic vs. Fitted and Forecast (2016-2027)",
    "First seasonal period excluded",
    "Helvetica",
    18,
)
fig.show()


**Observed vs. Fitted and Forecast (Excluding the Outlier)**

The second plot removes this reconstruction artifact, allowing a clearer view of the fitted values. Once the first seasonal point is excluded, the fitted series aligns closely with the observed data throughout the entire training window. The model reproduces the amplitude and timing of seasonal fluctuations effectively and tracks the year-over-year growth in solar generation.

The resulting forecast shows a realistic continuation of this pattern, with production rising seasonally each year and the forecasted peaks gradually increasing over the forecast horizon. The widening confidence intervals reflect greater forecast uncertainty farther into the future, but the central forecast path remains consistent with the underlying historical dynamics identified by the model.

___

### **6. Findings and Implications**
The forecast of solar photovoltaic energy cost in Spain over the next two years indicates a sustained increasing growth with a peak in energy demand during summer, reaching 6,678GWh in July 2026 and and 7,991GWh in July 2027. 

**Operational and Business Implications**

The forecasting results for Solar PV generation provide several practical insights that are directly relevant for energy planning and operational management. The model reveals a persistent upward trend in solar generation alongside a highly stable and predictable seasonal cycle. This means that operators can anticipate recurring annual peaks with confidence, enabling more effective scheduling of grid resources. **Higher expected production during spring and summer** months supports decisions related to storage optimization, load balancing, and renewable integration strategies. Conversely, **lower winter production** highlights periods where additional generation sources or increased storage discharge may be required.

The growing amplitude of the seasonal cycle in recent years further suggests that **the contribution of solar energy to the overall mix will continue strengthening**. This has implications for medium- to long-term resource planning, including the sizing of battery storage systems, curtailment strategies during high-production periods, and investments in transmission infrastructure to accommodate higher peak outputs. The model’s ability to capture these dynamics allows decision makers to anticipate operational stress points and design interventions to ensure reliability and efficiency in renewable energy integration.

**Methodological Implications**

Methodologically, the identification of SARIMA(0,1,1)(0,1,1)[12] as the best-performing model carries important implications for how Solar PV time-series data behave. **The absence of statistically meaningful autoregressive terms** suggests that Solar PV generation does not exhibit long memory once trend and seasonality are removed. Instead, **the dynamics are primarily explained by short-lived shocks** (through the non-seasonal MA component) and repeated annual seasonal disturbances (through the seasonal MA component). This confirms that **Solar PV generation is driven by immediate environmental fluctuations** rather than prolonged dependencies or inertia in the system.

**Limitations and Conditions**

Despite the strong performance of the model, several limitations must be acknowledged. First, the SARIMA framework assumes that the historical seasonal structure remains stable over time. If future years experience shifts in climate patterns, technological changes, or rapid expansions in installed capacity, the model may under- or overestimate peak production. Similarly, SARIMA models cannot explicitly represent external drivers such as **weather anomalies**, **cloud cover**, **temperature extremes**, or **policy-driven capacity expansions—factors** that can meaningfully influence solar generation.

The widening confidence intervals in the long-horizon forecast highlight growing uncertainty as projections extend beyond the training window. The model captures statistical patterns but **cannot anticipate structural breaks or nonlinear dynamics**. Additionally, residual diagnostics show mild deviations from normality, which do not invalidate the model but suggest that **extreme deviations may occasionally be underestimated.**